# Script de publicador

En este script ejecutaremos la simulación del centro comercial. Antes de empezar recuerda restaurar la base de datos suministrada (**sambil_db.sql**).

Primero importaremos las librerías necesarias y conectaremos con la base de datos (solo para compras) y con el broker MQTT.

In [ ]:
import sys
import paho.mqtt.client as mqtt
import psycopg2
import json
import random
import math
import time

# Agregamos la carpeta 'env' a nuestro path para importar modulos de ella
sys.path.insert(0, '../env')

# Este es el modulo que queremos importar. 
# Contiene informacion en un diccionario de 20 personas, solo el 80% tiene MAC (smartphone)
import personas_info

# Conectarse a la base de datos PostgreSQL
conn = psycopg2.connect(host='localhost', user='postgres', password='', dbname='')
cursor = conn.cursor()

# Crear instancia de cliente de MQTT
client = mqtt.Client()

# Conectar con broker local
client.connect("localhost", 1883, 60)

# Dejamos disponible al broker para recibir mensajes
client.loop_start()

predicciones = []

# Nodos asignados a entradas del centro comercial (no cambiar). Para nodos con dimension 5x5
entrada_nodo = {
    1: 1,
    2: 5,
    3: 16
}

También simularemos el funcionamiento de la inteligencia artificial de los accesos del centro comercial. Predice el sexo de una persona con un 97% de precisión. También tiene un margen de error aceptable para la edad.

In [ ]:
def IA_make_prediction(persona):
    prediccion = {'edad': None, 'sexo': None}
    if(random.random() > 0.03):
        prediccion['sexo'] = persona.sexo
    
    else:
        prediccion['sexo'] = persona.sexo_contrario()
    
    min_age = math.floor(persona.edad * 0.9)
    max_age = math.ceil(persona.edad * 1.1)
    prediccion['edad'] = random.randint(min_age, max_age)

    predicciones.append(prediccion)
    return prediccion

### Clase Persona

La clase persona contiene atributos y métodos que permiten a sus objetos interactuar con el centro comercial.

In [ ]:
class Persona:
    
    def __init__(self, nombre, ci, edad, sexo, mac):
        self.ci = ci
        self.nombre = nombre
        self.edad = edad
        self.sexo = sexo
        self.mac = mac
        self.io = False # In -> True | Out -> False
        self.io_tienda = False
        self.io_mesa = False
        self.tiempo_tienda = None
        self.nodo_actual = None
        self.nodo_anterior = None
        self.num_recorridos = 0
        self.acaba_de_comprar = 2
        self.tienda_actual = None
        self.mesa_actual = None

    # Simula una entrada o salida del centro comercial
    def acceso(self, entrada):
        msg = None
        AI_pred = {'edad': None, 'sexo': None}
        if((self.io == False) & (random.random() > 0.05)): # La IA tiene 95% de probabilidad de poder hacer una prediccion
            # Cuando la persona entra, la IA se activa
            AI_pred = IA_make_prediction(self)

        self.io = not self.io
        if(self.io ==  True):
            self.nodo_actual = entrada_nodo[entrada]
            self.nodo_anterior = entrada_nodo[entrada]
        else:
            num_recorridos = 0
        
        temp_mac = self.mac
        if(temp_mac == None):
            temp_mac = "[SIN MAC]"
        if(self.io):
            msg = "{} ha entrado al Sambil".format(temp_mac)
        else:
            msg = "{} ha salido del Sambil".format(temp_mac)

        msg_dict = {
            'message': msg,
            'sql_op': 0,
            'args': [entrada, self.io, self.mac, AI_pred['sexo'], AI_pred['edad']]
        }
        client.publish("Actividades", json.dumps(msg_dict), qos=0)
    
    # Simula movimiento de un nodo (posicion dentro del centro comercial) a otro
    def moverse(self):
        self.nodo_anterior = self.nodo_actual
        nodo_nuevo = self.nodo_actual + random.choice([1,4,5,6,-1,-4,-5,-6])
        while(not (1 <= nodo_nuevo <= 20)):
            nodo_nuevo = self.nodo_actual + random.choice([1,4,5,6,-1,-4,-5,-6])
        self.nodo_actual = nodo_nuevo
        temp_mac = self.mac
        if(temp_mac == None):
            temp_mac = "[SIN MAC]"
        msg = "{} se ha movido de {} a {}".format(temp_mac, self.nodo_anterior, self.nodo_actual)
        msg_dict = {
            'message': msg,
            'sql_op': 1,
            'args': [self.nodo_actual, self.mac]
        }
        client.publish("Actividades", json.dumps(msg_dict), qos=0)
        
    # Simula una compra (no figura como operacion MQTT)
    def comprar(self, id_tienda, monto):
        try:
            cursor.callproc('compra', [self.ci, self.nombre, id_tienda, monto, self.mac])
            conn.commit()
        except psycopg2.errors.RaiseException as e:
            print('Se ha tratado de comprar con una MAC que no está en el centro comercial')
        
        temp_mac = self.mac
        if(temp_mac == None):
            temp_mac = "[SIN MAC]"
        print("{} ha comprado por un monto de {} en la tienda {}".format(temp_mac, monto, id_tienda))

    # Simula la entrada o salida de una tienda
    def tienda(self, id_tienda):
        msg = None
        self.io_tienda = not self.io_tienda
        temp_mac = self.mac
        if(temp_mac == None):
            temp_mac = "[SIN MAC]"
        if(self.io_tienda == False):
            self.tienda_actual = None
            msg = "{} ha salido de la tienda {}".format(temp_mac, id_tienda)
        else:
            msg = "{} ha entrado a la tienda {}".format(temp_mac, id_tienda)
        
        msg_dict = {
            'message': msg,
            'sql_op': 2,
            'args': [id_tienda, not self.io_tienda, self.mac]
        }
        client.publish("Actividades", json.dumps(msg_dict), qos=1)
    
    # Simula el uso o desuso de una mesa del centro comercial
    def mesa(self, id_mesa):
        msg = None
        self.io_mesa = not self.io_mesa
        temp_mac = self.mac
        if(temp_mac == None):
            temp_mac = "[SIN MAC]"
        if(self.io_mesa == False):
            self.mesa_actual = None
            msg = "{} se ha parado de la mesa {}".format(temp_mac, id_mesa)
        else:
            msg = "{} se ha sentado en la mesa {}".format(temp_mac, id_mesa)

        msg_dict = {
            'message': msg,
            'sql_op': 3,
            'args': [id_mesa, not self.io_mesa, self.mac]
        }
        client.publish("Actividades", json.dumps(msg_dict), qos = 2)

    # Ejecuta una decision de la persona, se le pasa como argumento una probabilidad de escoger
    def decision(self, probabilidad):
        return random.random() < probabilidad

    # Se ejecuta por cada persona en cada iteración. Es el arbol de comportamiento.
    def update(self):   

        if(self.io_tienda == True):
            if(self.decision(0.5)): # 50% de probabilidad de comprar
                self.comprar(self.tienda_actual, random.randrange(5000, 1000000))
            self.tienda(self.tienda_actual)
            self.moverse()
            return None

        if(self.io_mesa == True):
            if(self.decision(0.5)):
                self.mesa(self.mesa_actual)
                self.moverse()
                return None
        
        if(self.nodo_actual != None):
            if(self.nodo_actual % 2 == 0):
                if(self.decision(0.65)):
                    self.tienda_actual = int(self.nodo_actual / 2)
                    self.tienda(self.tienda_actual)
                else:
                    self.moverse()
                return None
            elif(self.nodo_actual % 5 == 0):
                if(self.decision(0.3)):
                    self.mesa_actual = int(self.nodo_actual / 5)
                    self.mesa(self.mesa_actual)
                    return None
                else:
                    self.moverse()
        
        if(self.io == False):
            self.acceso(random.choice([1,2,3]))
            return None
        elif((self.nodo_actual == 1) | (self.nodo_actual == 5) | (self.nodo_actual == 16)):
            if(self.decision(0.2)):
                self.acceso(list(entrada_nodo.keys())[list(entrada_nodo.values()).index(self.nodo_actual)])
                return None
            else:
                self.moverse()
                return None
            
        self.moverse()

    # Simula error de sexo de la IA
    def sexo_contrario(self):
        if(self.sexo == "M"):
            return "F"

        return "M"

La información de las personas que intervendrán se almacena en `personas_data`. Iteraremos sobre esta variable para crear un arreglo de objetos de la clase `Persona`.

In [ ]:
personas_data = personas_info.payload

personas = []

for info in personas_data:
    personas.append(Persona(info["nombre"], info["cedula"], info["edad"], info["sexo"], info["mac"]))

Con la siguiente función simularemos el comportamiento de las personas dentro del centro comercial, llamando repetidas veces al método `update()`, que da una nueva tarea a un objeto de Persona.

Puedes ver el número de iteraciones como las horas que pasan en la simulación.

Al ejecutarse aparecerá en consola únicamente las compras realizadas en cada iteración, si deseas ver todo lo demás que ocurre en el centro comercial, ejecuta el Notebook **Subscriptor** y déjalo correr.

**NOTA**: este script solo trabaja con la base de datos para insertar compras, ya que no figuran como actividades captadas por sensores, estas actividades sensoriales son registradas por el script del Notebook **Subscriptor**.

In [ ]:
def simular(iteraciones):
    for i in range(0, iteraciones):
        print("------------------------ITERACION {}------------------------".format(i+1))
        for persona in personas:
            persona.update()
        time.sleep(3)
    print("LA SIMULACION HA FINALIZADO")

Ya podemos simular el centro comercial.

In [ ]:
simular(10)